# Fuzzy Wuzzy - Levenshtein Distance

- string metric to measure the similarity between two strings
- the Levenshtein Distance is the min. number of single-character edits (insertions, deletions, substitutions) required to change a word into another (aka the Edit Distance)


*Things to Keep in Mind*
- your computer is dumb -- it doesn't know that "S." refers to "SOUTH" or that "CLFK" refers to "CLEAR FORK".  Manual spot checking is advised
- in a dataframe of fuzzy wuzzy matching, after a certain fuzzy wuzzy ratio score (ex. >95), we can assume everything is correctly matched, but there will often be a few correct matches at much lower scores due to examples seen in the first bullet (S. vs. SOUTH with a score of 29)


# Import packages

In [1]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np
import os

import warning
warnings

C:\Users\epease\.conda\envs\geo_env\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Fuzzy Wuzzy Ratio Examples

In [2]:
fuzz.ratio("Python is fun", "Python is fun!")

96

In [3]:
fuzz.ratio("Python is fun", "Python is fun!")

96

In [4]:
fuzz.ratio("Hello there", "General Kenobi")

32

In [5]:
fuzz.ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

91

# Extract best choice(s) from list - use extractOne to only return one match

In [6]:
choices = ["Eric", "Hayden", "Emily", "Parker"]

process.extract("erik", choices, limit=1)

[('Eric', 75)]

In [7]:
choices = ["WFCP S.", "WFCP S", "WLF S", "WDFD S"]

process.extract("WOLFCAMP S", choices, limit=2)

[('WFCP S.', 86), ('WFCP S', 86)]

In [8]:
choices = ["CL.FK.", "CLEAR FK. S.", "CLFK S."]
print(choices)

process.extract("CLEAR FORK SOUTH", choices, limit=3)

['CL.FK.', 'CLEAR FK. S.', 'CLFK S.']


[('CLEAR FK. S.', 74), ('CLFK S.', 60), ('CL.FK.', 54)]

# Real world examples

In [9]:
# Load data
df = pd.read_csv(os.path.join("..", "data", "H2S.csv"))
df.head()

,DISTRICT,COUNTY,AVG CONCEN,Field,FIELD,Formation,FORMATION,FORM_ID,FORM_AGE,ABRV,LONG_NAME,Longitude,Latitude
0,7C,CROCKETT,96000,MIDWAYLANE,MIDWAY LANE,1050SD.,NaN,NaN,NaN,NaN,NaN,-101.300109,30.900531
1,7C,CROCKETT,96000,MIDWAYLANE,MIDWAY LANE,1050SD.,NaN,NaN,NaN,NaN,NaN,-101.300109,30.900531
2,7C,CROCKETT,96000,MIDWAYLANE,MIDWAY LANE,1050SD.,NaN,NaN,NaN,NaN,NaN,-101.300109,30.900531
3,7C,CROCKETT,96000,MIDWAYLANE,MIDWAY LANE,1050SD.,NaN,NaN,NaN,NaN,NaN,-101.300109,30.900531
4,7C,CROCKETT,96000,MIDWAYLANE,MIDWAY LANE,1050SD.,NaN,NaN,NaN,NaN,NaN,-101.300109,30.900531


In [10]:
# Choose columns to include
df = df[["AVG CONCEN", "Field", "FIELD"]]

In [11]:
# Shuffled the data for this example 
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,AVG CONCEN,Field,FIELD
0,58571,"SEAGRAVES,SOUTH",SEAGRAVES SOUTH
1,21879,LEVELLAND,LEVELLAND
2,118,MCFARLAND,MCFARLAND
3,492,TEXEL,TEXEL
4,61004,DEEPROCK,DEEP ROCK


In [12]:
# Only select the last 50 rows for this example
df_filt = df.iloc[18300:, :] 
df_filt.head()

,AVG CONCEN,Field,FIELD
18300,7053,BLOCKD,BLOCK D
18301,4904,"DOLLARHIDE,EAST",DOLLARHIDE EAST
18302,8000,ROBLAW,ROBLAW
18303,1200,WHEELER,WHEELER
18304,579,PECOSVALLEY,PECOS VALLEY


In [16]:
# Create an empty list "scores", in which to append the fuzzy wuzzy ratios
scores = []

# for loop through dataframe to calculate the ratio between two columns "Field" and "FIELD",
for index, row in df_filt.iterrows():
    score = fuzz.ratio(row["Field"], row["FIELD"]) 
    scores.append(score) # append the fuzzywuzzy 'score' to the list 'scores'

In [17]:
# convert 'scores' list into a numpy array to allow for easier transformation back into the dataframe
scores = np.array(scores)
df_filt["FUZZY_Field_Score"] = scores

C:\Users\epease\.conda\envs\geo_env\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
# print whole dataframe with FUZZYSCORE on the right end
df_filt

,AVG CONCEN,Field,FIELD,FUZZY_Field_Score
18300,7053,BLOCKD,BLOCK D,92
18301,4904,"DOLLARHIDE,EAST",DOLLARHIDE EAST,93
18302,8000,ROBLAW,ROBLAW,100
18303,1200,WHEELER,WHEELER,100
18304,579,PECOSVALLEY,PECOS VALLEY,96
18305,217,PEGASUS,PEGASUS,100
18306,6751,PUTNAM,PUTNAM,100
18307,1874,SPRABERRY,SPRABERRY,100
18308,22010,DOVER,DOVER,100
18309,30878,FULLERTON,FULLERTON,100
